<a href="https://colab.research.google.com/github/PRR-aiexp/tinymlexp/blob/main/tinymlexp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.7/774.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00


In [3]:
import mlflow

mlflow.set_tracking_uri("file:/content/mlruns")
mlflow.set_experiment("TinyML_MNIST")


/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2025/12/26 19:49:37 INFO mlflow.tracking.fluent: Experiment with name 'TinyML_MNIST' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/319141353693039943', creation_time=1766778577297, experiment_id='319141353693039943', last_update_time=1766778577297, lifecycle_stage='active', name='TinyML_MNIST', tags={}>

In [7]:
import tensorflow as tf
import mlflow.tensorflow as mlf_tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train/255.0
y_train = y_train/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(32,activation='relu'),
                                    tf.keras.layers.Dense(10,activation='softmax')
                                    ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


with mlflow.start_run():
  history = model.fit(x_train,y_train,epochs=3, validation_split=0.1)
  loss, acc = model.evaluate(x_test,y_test)
  mlflow.log_metric("test accuracy",acc)
  mlflow.log_metric("test loss",loss)
  mlf_tf.log_model(model,"keras_mnist_model")




/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0990 - loss: 0.0566 - val_accuracy: 0.0978 - val_loss: 4.5815e-05
Epoch 2/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1009 - loss: 4.0680e-05 - val_accuracy: 0.0978 - val_loss: 9.7592e-06
Epoch 3/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1000 - loss: 1.0125e-05 - val_accuracy: 0.0978 - val_loss: 2.9819e-06
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0924 - loss: 5430.3130


2025/12/26 19:57:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/26 19:57:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


In [8]:
def rep_data():
  for cnt in range(100):
    yield [x_train[cnt:cnt+1].astype("float32")]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep_data
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()
with open("mnist_tinyml.tflite","wb") as f:
  f.write(tflite_model)

mlflow.log_artifact("mnist_tinyml.tflite")


Saved artifact at '/tmp/tmp1he5tjbi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137168235976016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137168235976976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137168235976400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137168235976784: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
